In [1]:
import pandas as pd
import numpy as np

In [2]:
temp_df = pd.read_csv("IMDB Dataset.csv")

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df.duplicated().sum()

17

In [9]:
df = df.drop_duplicates()

In [10]:
df.duplicated().sum()

0

In [11]:
### Basic Preprocessing
### Remove Tags, Lowercase, Stopwords

import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [12]:
df['review'] = df['review'].apply(remove_tags)

In [13]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [15]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [16]:
from nltk.corpus import stopwords
sw_list = stopwords.words('english')
print(sw_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [18]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [20]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [21]:
story = []

for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [23]:
story[:3]

[['one', 'reviewers', 'mentioned', 'watching', 'oz', 'episode', 'hooked'],
 ['right',
  'exactly',
  'happened',
  'me',
  'the',
  'first',
  'thing',
  'struck',
  'oz',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'go'],
 ['trust', 'me', 'show', 'faint', 'hearted', 'timid']]

In [24]:
## Training model on our own dataset for better embeddings.
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [25]:
model.build_vocab(story)

In [ ]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs) 

(5876398, 6212140)

In [ ]:
len(model.wv.index_to_key) ## we have these many words in our vocabulary and we have vectorized them as well.

31845

In [29]:
## Now we want vectors for the reviews which are sentences so we will use concept of Averageword2vec
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [30]:
document_vector(df['review'].values[0])

array([-0.20305902,  0.49906957,  0.12693292,  0.22745223, -0.15063366,
       -0.6448775 ,  0.25666666,  0.9445885 , -0.31249666, -0.27709368,
       -0.31008425, -0.4911613 ,  0.07569393,  0.08137266,  0.17175563,
       -0.06724931, -0.01080978, -0.33791623, -0.0899639 , -0.57930946,
        0.02415334,  0.27332094,  0.08463015, -0.29184374, -0.29635236,
       -0.03184298, -0.3180967 , -0.03998353, -0.292735  ,  0.01336009,
        0.29019076, -0.02860915,  0.17960203, -0.23869507, -0.12519005,
        0.47399563,  0.10242423, -0.42512098, -0.2627911 , -0.7332875 ,
        0.10548101, -0.28501105,  0.07501391, -0.0759933 ,  0.48664287,
       -0.13517696, -0.25684753, -0.05113238,  0.11451073,  0.35138017,
        0.05257775, -0.37928355, -0.39490965, -0.06364234, -0.09091186,
        0.20483404,  0.23254122, -0.0179895 , -0.28331625,  0.11991054,
        0.06816767,  0.09733047,  0.01428403, -0.11499915, -0.45787197,
        0.29783723,  0.04302365,  0.11354747, -0.34113222,  0.26

In [31]:
from tqdm import tqdm ## tqdm is only for the progress bar.
X =[]
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [06:50<00:00, 24.29it/s]


In [ ]:
len(X) ## X is a list of 9983 elements each being a 1 D array of 100 dimensions

9983

In [37]:
X = np.array(X)

In [38]:
X.shape

(9983, 100)

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(df['sentiment'])

In [39]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [40]:
X_train.shape,X_test.shape

((7986, 100), (1997, 100))

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.7646469704556835
[[716 236]
 [234 811]]
